In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import NuSVR, LinearSVR
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [2]:
train_data_path = './dataset/train_data.csv'
train_data = pd.read_csv(train_data_path)
train_data.drop(columns=['Name'], inplace=True)
X = train_data.to_numpy(dtype='float64')
X.shape
y = X[:, -1]
X = X[:, :-1]
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10)


(2090, 9)
(2090,)


## Linear Regression

In [3]:
LR = LinearRegression()
LR.fit(X_train, y_train)
prediction = LR.predict(X_test)
error = prediction - y_test
MSE = np.sum(np.power(error, 2)) / error.shape[0]
print(MSE)

0.044309591930424955


## Neural Network

In [6]:
MLP = MLPRegressor(hidden_layer_sizes=200, batch_size=50, verbose=True, learning_rate='adaptive', learning_rate_init=1e-6, max_iter=5000)
MLP.fit(X_train, y_train)
prediction = MLP.predict(X_test)
error = prediction - y_test
MSE = np.sum(np.power(error, 2)) / error.shape[0]
print(MSE)

Iteration 1, loss = 41184.00155029
Iteration 2, loss = 40797.14245590
Iteration 3, loss = 40413.04776730
Iteration 4, loss = 40031.77036351
Iteration 5, loss = 39653.31003730
Iteration 6, loss = 39277.61576489
Iteration 7, loss = 38904.66515443
Iteration 8, loss = 38534.46489620
Iteration 9, loss = 38167.00282942
Iteration 10, loss = 37802.15151870
Iteration 11, loss = 37440.04366004
Iteration 12, loss = 37080.55685618
Iteration 13, loss = 36723.74674167
Iteration 14, loss = 36369.49797705
Iteration 15, loss = 36017.87970402
Iteration 16, loss = 35668.86368285
Iteration 17, loss = 35322.35383388
Iteration 18, loss = 34978.35788876
Iteration 19, loss = 34636.85943058
Iteration 20, loss = 34297.79438108
Iteration 21, loss = 33961.16827942
Iteration 22, loss = 33626.92006447
Iteration 23, loss = 33295.16571092
Iteration 24, loss = 32965.79454685
Iteration 25, loss = 32638.76825780
Iteration 26, loss = 32314.08028447
Iteration 27, loss = 31991.74194621
Iteration 28, loss = 31671.63417036
I

## SVM

In [9]:
NSVR = NuSVR(verbose=True, kernel='poly', degree=10, C=10)
NSVR.fit(X_train, y_train)
prediction = NSVR.predict(X_test)
error = prediction - y_test
MSE = np.sum(np.power(error, 2)) / error.shape[0]
print(MSE)


[LibSVM]0.04963080010557844


In [10]:
LSVR = LinearSVR(verbose=False, C=10)
LSVR.fit(X_train, y_train)
prediction = LSVR.predict(X_test)
error = prediction - y_test
MSE = np.sum(np.power(error, 2)) / error.shape[0]
print(MSE)

0.10974840605154977


d:\anaconda3\envs\ai-final\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\anaconda3\envs\ai-final\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Principal Component Analysis (PCA)

In [14]:
PCAmodel = PCA()
PCAmodel.fit(X)
PCAmodel.score(X)

AttributeError: 'PCA' object has no attribute 'feature_names_in_'